In [1]:
import numpy as np

In [2]:
def gen_base(H, V, dx, dy, wavelength, f):

    """

    gen_base - Function to generate a structured grid (meshgrid) and related parameters.

    Parameters:

        H (int): Number of horizontal grid points (columns).

        V (int): Number of vertical grid points (rows).

        dx (float): Grid spacing in the horizontal direction (x-axis).

        dy (float): Grid spacing in the vertical direction (y-axis).

        wavelength (float): Wavelength of interest (not used directly in this function but passed for flexibility).

        f (float): Focal length (not used directly in this function but passed for flexibility).

    Returns:

        dict: A dictionary containing spatial and frequency domain mesh grids, grid spacings, and polar coordinates.

    """

    # Generate the spatial domain grid points for y (vertical axis)

    y = np.linspace(-(V / 2), (V / 2) - 1, V) * dy  # Scaled by vertical spacing (dy)

    # Generate the spatial domain grid points for x (horizontal axis)

    x = np.linspace(-(H / 2), (H / 2) - 1, H) * dx  # Scaled by horizontal spacing (dx)

    # Generate the frequency domain grid points for fy (vertical frequencies)

    fy =  y/ (wavelength *f)  # Normalized by grid spacing and size np.linspace(-(V / 2), (V / 2) - 1, V)*dy

    # Generate the frequency domain grid points for fx (horizontal frequencies)

    fx = x/ (wavelength * f)  # Normalized by grid spacing and size  np.linspace(-(H / 2), (H / 2) - 1, H)*dx 

    # Create the spatial domain meshgrid (Cartesian coordinates)

    X, Y = np.meshgrid(x, y)

    # Create the frequency domain meshgrid

    FX, FY = np.meshgrid(fx, fy)

    # Convert spatial domain coordinates to polar coordinates

    r = np.sqrt(X**2 + Y**2)  # Radius

    phi = np.arctan2(Y, X)    # Angle in radians

    #Normalization factor for the Fourier Transform

    Normalization = np.sqrt(H*V)*(dx * dy)

    # Store all grids and parameters in a dictionary

    base = {

        "y": y,      # Vertical grid points

        "x": x,      # Horizontal grid points

        "dx": dx,    # Grid spacing in x

        "dy": dy,    # Grid spacing in y

        "H": H,      # Number of horizontal points

        "V": V,      # Number of vertical points

        "X": X,      # Meshgrid for x-coordinates (spatial domain)

        "Y": Y,      # Meshgrid for y-coordinates (spatial domain)

        "FX": FX,    # Meshgrid for fx-coordinates (frequency domain)

        "FY": FY,    # Meshgrid for fy-coordinates (frequency domain)

        "phi": phi,  # Polar coordinate angles

        "r": r,       # Polar coordinate radii

        "Normalization": Normalization #Normalization factor for the Fourier Transform

    }


    return base

Mesh = gen_base(3, 3, 8e-6, 8e-6, 633e-9,20e-3) # H V dx dy wavelength f